# Role Prompting

Role prompting involves asking the LLM to assume a specific role or identity before performing a given task, such as acting as a copywriter. This can help guide the model's response by providing a context or perspective for the task. To work with role prompts, you could iteratively:

- Specify the role in your prompt, e.g., "As a copywriter, create some attention-grabbing taglines for AWS services."
- Use the prompt to generate an output from an LLM.
- Analyze the generated response and, if necessary, refine the prompt for better results.

In [1]:
%pip install -qU langchain-openai
%pip install -qU langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata
from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
# Get the API key from Colab's userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

# Set it as an environment variable
os.environ["OPENAI_API_KEY"] = openai_api_key
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""
prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"}

chain = prompt | llm

response = chain.invoke(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response.content)

Theme: interstellar travel
Year: 3030
AI-generated song title: "Galactic Odyssey: Echoes of the Stars"


This is a good prompt for several reasons:

- **Clear instructions**: The prompt is phrased as a clear request for help in generating a song title, and it specifies the context: "As a futuristic robot band conductor." This helps the LLM understand that the desired output should be a song title related to a futuristic scenario.
- **Specificity**: The prompt asks for a song title that relates to a specific theme and a specific year, "{theme} in the year {year}." This provides enough context for the LLM to generate a relevant and creative output. The prompt can be adapted for different themes and years by using input variables, making it versatile and reusable.
- **Open-ended creativity**: The prompt allows for open-ended creativity, as it doesn't limit the LLM to a particular format or style for the song title. The LLM can generate a diverse range of song titles based on the given theme and year.
- **Focus on the task**: The prompt is focused solely on generating a song title, making it easier for the LLM to provide a suitable output without getting sidetracked by unrelated topics.

These elements help the LLM understand the user's intention and generate a suitable response.

## Few Shot Prompting
Few Shot Prompting In the next example, the LLM is asked to provide the emotion associated with a given color based on a few examples of color-emotion pairs:



In [8]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""
example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="purple")

prompt=PromptTemplate(template=formatted_prompt, input_variables=[])
chain = prompt | llm

# Run the Runnable to get the AI-generated emotion associated with the input color
response = chain.invoke({})

print(response.content)

Color: purple  
Emotion: creativity


This prompt provides **clear instructions** and few-shot examples to help the model understand the task.

## Chain Prompting
Chain Prompting refers to the practice of chaining consecutive prompts, where the output of a previous prompt becomes the input of the successive prompt.

To use chain prompting with LangChain, you could:

- Extract relevant information from the generated response.
- Use the extracted information to create a new prompt that builds upon the previous response.
- Repeat steps as needed until the desired output is achieved.

`PromptTemplate` class makes constructing prompts with dynamic inputs easier. This is useful when creating a prompt chain that depends on previous answers.

In [13]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity? Give the name only and stop there.
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the Runnable for the first prompt
chain_question = prompt_question | llm

response_question = chain_question.invoke({})

# Extract the scientist's name from the response
scientist = response_question.content.strip()

# Create the Runnable for the second prompt
chain_fact = prompt_fact | llm

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the Runnable for the second prompt
response_fact = chain_fact.invoke(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact.content)

Scientist: Einstein
Fact: Einstein's theory of general relativity, published in 1915, is a fundamental theory of gravitation that describes gravity not as a force, but as a curvature of spacetime caused by mass and energy. According to this theory, massive objects like planets and stars warp the fabric of spacetime around them, and this curvature affects the motion of other objects, causing them to follow curved paths. General relativity has profound implications for our understanding of the universe, including the behavior of light around massive bodies, the expansion of the universe, and the prediction of phenomena such as black holes and gravitational waves. It has been confirmed through numerous experiments and observations, fundamentally changing our understanding of gravity and the cosmos.


**Bad Prompt Example**

In [15]:
# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something interesting about {scientist}.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = prompt_question |llm

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.invoke({})

# Extract the scientist's name from the response
scientist = response_question.content.strip()

# Create the LLMChain for the second prompt
chain_fact = prompt_fact | llm

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.invoke(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact.content)

Scientist: The famous scientist who developed the theory of general relativity is Albert Einstein.
Fact: One interesting aspect of Albert Einstein's life is that he was not only a brilliant physicist but also a passionate advocate for civil rights. Despite being a prominent figure in the scientific community, Einstein used his platform to speak out against racism and discrimination. He was a member of the NAACP and developed a close friendship with civil rights leader W.E.B. Du Bois. Einstein often expressed his belief in the importance of equality and justice, famously stating, "Racism is a disease of white people." His commitment to social issues extended beyond his scientific work, highlighting his belief that scientists have a responsibility to engage with the world and advocate for positive change.


## Chain of Thought Prompting
Chain of Thought Prompting (CoT) is a technique developed to encourage large language models to explain their reasoning process, leading to more accurate results. By providing few-shot exemplars demonstrating the reasoning process, the LLM is guided to explain its reasoning when answering the prompt. This approach has been found effective in improving results on tasks like arithmetic, common sense, and symbolic reasoning.

In the context of LangChain, CoT can be beneficial for several reasons. First, it can help break down complex tasks by assisting the LLM in decomposing a complex task into simpler steps, making it easier to understand and solve the problem. This is particularly useful for calculations, logic, or multi-step reasoning tasks. Second, CoT can guide the model through related prompts, helping generate more coherent and contextually relevant outputs. This can lead to more accurate and useful responses in tasks that require a deep understanding of the problem or domain.

There are some limitations to consider when using CoT. One limitation is that it has been found to yield performance gains only when used with models of approximately 100 billion parameters or larger; smaller models tend to produce illogical chains of thought, which can lead to worse accuracy than standard prompting. Another limitation is that CoT may not be equally effective for all tasks. It has been shown to be most effective for tasks involving arithmetic, common sense, and symbolic reasoning. For other types of tasks, the benefits of using CoT might be less pronounced or even counterproductive.

**Tips for Effective Prompt Engineering**

- **Be specific with your prompt**: Provide enough context and detail to guide the LLM toward the desired output.
- **Force conciseness when needed**.
- **Encourage the model to explain its reasoning**: This can lead to more accurate results, especially for complex tasks.

Keep in mind that prompt engineering is an iterative process, and it may require several refinements to obtain the best possible answer. As LLMs become more integrated into products and services, the ability to create effective prompts will be an important skill to have.

In [17]:
# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the Runnable for the few-shot prompt template
chain = few_shot_prompt_template | llm

# Define the user query
user_query = "What are some tips for improving communication skills?"

# Run the Runnable for the user query
response = chain.invoke({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response.content)

User Query: What are some tips for improving communication skills?
AI Response: Practice active listening, be clear and concise in your messages, and seek feedback to refine your approach. Engaging in conversations regularly and being open to different perspectives can also enhance your skills.


This prompt:

- **Provides a clear context in the prefix**: The prompt states that the AI is a life coach providing insightful and practical advice. This context helps guide the AI's responses and ensures they align with the intended purpose.
- **Uses examples** that demonstrate the AI's role and the **type of responses** it generates: By providing relevant examples, the AI can better understand the style and tone of the responses it should produce. These examples serve as a reference for the AI to generate similar responses that are consistent with the given context.
- Separates examples and the actual query: This **allows the AI to understand the format it should follow**, ensuring a clear distinction between example conversations and the user's input. This separation helps the AI to focus on the current query and respond accordingly.
- Includes a clear suffix that indicates where the user's input goes and where the AI should provide its response: The suffix acts as a cue for the AI, showing where the user's query ends and the AI's response should begin. This structure helps maintain a **clear and consistent format** for the generated responses.
By using this well-structured prompt, the AI can understand its role, the context, and the expected response format, leading to more accurate and useful outputs.